In [1]:
import math
import json
import warnings
import numpy as np
import pandas as pd
import csv
import copy
import matplotlib.pyplot as plt

from collections import OrderedDict
from IPython.display import display, HTML

from cryptorandom.cryptorandom import SHA256, int_from_hash
from cryptorandom.sample import sample_by_index

from Audit import Audit, Assertion, Assorter, Contest, CVR, Stratum
from NonnegMean import NonnegMean
from Dominion import Dominion
from Hart import Hart

In [3]:
audit = Audit.from_dict({
         'seed':           12345678901234567890,
         'sim_seed':       314159265,
         'cvr_file':       '/Users/amanda/Downloads/cvrs.zip', 
         #'cvr_file':       '/Users/Jake/Desktop/OC CVRs/2022/cvrs.zip', 
         #'manifest_file':  'Data/OC_mock_manifest_detailed.xlsx',
         'manifest_file': 'Data/OC_nov22_full_manifest.xlsx',
         #'manifest_file': 'tests/Data/Hart_manifest.xlsx',
         'sample_file':    '',
         'mvr_file':       '',
         'log_file':       'Data/OC_example_log.json',
         'quantile':       0.8,
         'error_rate_1': 0,
         'error_rate_2': 0,
         #'error_rate_1':   0.001,
         #'error_rate_2':   0.0001,
         'reps':           100,
         'strata':         {'stratum_1': {'max_cards':   1989416, 
                                          'use_style':   True,
                                          'replacement': False,
                                          'audit_type':  Audit.AUDIT_TYPE.BALLOT_COMPARISON,
                                          'test':        NonnegMean.alpha_mart,
                                          'estimator':   NonnegMean.optimal_comparison,
                                          'test_kwargs': {}
                                         }
                           }
        })

# find upper bound on total cards across strata
audit.max_cards = np.sum([s.max_cards for s in audit.strata.values()])

In [ ]:
%%time
cvr_list = Hart.read_cvrs_zip(audit.cvr_file)

In [ ]:
votes = CVR.tabulate_votes(cvr_list)
styles = CVR.tabulate_styles(cvr_list)
cards = CVR.tabulate_cards_contests(cvr_list)

In [ ]:
#make dict of all contests in CVRs
contests = Contest.from_cvr_list(audit, votes, cards, cvr_list)

In [ ]:
# Create array of vote for 2 contests TO DO
## Data from https://ocvote.gov/data/election-results-archives
vote_for_two = [
    'CITY OF ALISO VIEJO\nMember, City Council',
    'CITY OF IRVINE\nMember, City Council',
    'CITY OF LAGUNA NIGUEL\nMember, City Council',
    'CITY OF LAGUNA WOODS\nMember, City Council',
    'CITY OF LA HABRA\nMember, City Council, Full Term',
    'CITY OF LA PALMA\nMember, City Council',
    'CITY OF YORBA LINDA\nMember, City Council',
    'ROSSMOOR COMMUNITY SERVICES DISTRICT\nDirector'
               ]
# Create array of vote for 3 contests TO DO
vote_for_three = [
    'YORBA LINDA WATER DISTRICT\nDirector',
    'MOULTON NIGUEL WATER DISTRICT\nDirector',
    'MIDWAY CITY SANITARY DISTRICT\nDirector',
    'CITY OF LAGUNA HILLS\nMember, City Council, Full Term',
    'CITY OF FOUNTAIN VALLEY\nMember, City Council',
    'CITY OF CYPRESS\nMember, City Council',
    'HUNTINGTON BEACH UNION HIGH SCHOOL DISTRICT\nGoverning Board Member',
    'FOUNTAIN VALLEY SCHOOL DISTRICT\nGoverning Board Member',
    'LA HABRA CITY SCHOOL DISTRICT\nGoverning Board Member',
    'OCEAN VIEW SCHOOL DISTRICT\nGoverning Board Member',
    'CITY OF BREA\nMember, City Council',
    'CITY OF LAGUNA BEACH\nMember, City Council',
    'RANCHO SANTA MARGARITA\nMember, City Council',
    'CITY OF SAN CLEMENTE\nMember, City Council',
    'CITY OF VILLA PARK\nMember, City Council',
    'SILVERADO-MODJESKA RECREATION AND PARK DISTRICT\nDirector'
                 ]
# Update dictionary with correct n_winners
for con in vote_for_two:
    contests[con].n_winners = 2
    
for con in vote_for_three:
    contests[con].n_winners = 3

contests['CITY OF HUNTINGTON BEACH\nMember, City Council'].n_winners = 4 
# TO DO: UPDATE WINNERS FOR THESE CONTESTS

In [ ]:
contests['CITY OF ALISO VIEJO\nMember, City Council'].winner = ['TIFFANY ACKLEY','MAX DUNCAN']
contests['CITY OF IRVINE\nMember, City Council'].winner = ['LARRY AGRAN','KATHLEEN TRESEDER']
contests['CITY OF LAGUNA NIGUEL\nMember, City Council'].winner = ['STEPHANIE ODDO','RAY GENNAWEY']
contests['CITY OF LAGUNA WOODS\nMember, City Council'].winner = ['CAROL MOORE','ANNIE MCCARY']
contests['CITY OF LA HABRA\nMember, City Council, Full Term'].winner = ['JOSE MEDRANO','JAMES "JIM" GOMEZ']
contests['CITY OF LA PALMA\nMember, City Council'].winner = ['NITESH PATEL','JANET K. CONKLIN']
contests['CITY OF YORBA LINDA\nMember, City Council'].winner = ['CARLOS RODRIGUEZ','JANICE LIM']
contests['ROSSMOOR COMMUNITY SERVICES DISTRICT\nDirector'].winner = ['MICHAEL MAYNARD','JO SHADE']

contests['YORBA LINDA WATER DISTRICT\nDirector'].winner = ['PHILLIP (PHIL) D. HAWKINS','THOMAS LINDSEY','BRETT R. BARBRE']
contests['MOULTON NIGUEL WATER DISTRICT\nDirector'].winner = ['DIANE RIFKIN', 'WILLIAM "BILL" MOORHEAD', 'DONALD R. FROELICH']
contests['MIDWAY CITY SANITARY DISTRICT\nDirector'].winner = ['ANDREW NGUYEN','SERGIO CONTRERAS','MARK NGUYEN']
contests['SILVERADO-MODJESKA RECREATION AND PARK DISTRICT\nDirector'].winner = ['LAURIE MARTZ','BRITTNEY KUHN','TED WRIGHT']
contests['CITY OF VILLA PARK\nMember, City Council'].winner = ['CRYSTAL K. MILES','NICOL JONES','JORDAN WU']
contests['CITY OF SAN CLEMENTE\nMember, City Council'].winner = ['VICTOR CABRAL','MARK ENMEIER','STEVEN "STEVE" KNOBLOCK']
contests['RANCHO SANTA MARGARITA\nMember, City Council'].winner = ['ANNE FIGUEROA', 'BRAD MCGIRR', 'JERRY HOLLOWAY']
contests['CITY OF LAGUNA HILLS\nMember, City Council, Full Term'].winner = ['ERICA PEZOLD', 'DAVID WHEELER','JOSHUA SWEENEY']
contests['CITY OF LAGUNA BEACH\nMember, City Council'].winner = ['ALEX ROUNAGHI','SUE KEMPF','MARK S. ORGILL']
contests['CITY OF FOUNTAIN VALLEY\nMember, City Council'].winner = ['JIM CUNNEEN','KIM CONSTANTINE','PATRICK HARPER']
contests['CITY OF CYPRESS\nMember, City Council'].winner = ['DAVID BURKE','SCOTT MINIKUS','BONNIE PEAT']
contests['CITY OF BREA\nMember, City Council'].winner = ['STEVEN C. VARGAS','BLAIR STEWART','CECILIA HUPP']
contests['HUNTINGTON BEACH UNION HIGH SCHOOL DISTRICT\nGoverning Board Member'].winner = ['DUANE DISHNO', 'BONNIE CASTREY', 'DIANA LEE CAREY']
contests['FOUNTAIN VALLEY SCHOOL DISTRICT\nGoverning Board Member'].winner = ['SANDRA CRANDALL', 'DENNIS COLE', 'PHU NGUYEN']
contests['LA HABRA CITY SCHOOL DISTRICT\nGoverning Board Member'].winner = ['ADAM ROGERS','CYNTHIA AGUIRRE','EMILY PRUITT']
contests['OCEAN VIEW SCHOOL DISTRICT\nGoverning Board Member'].winner = ['PATRICIA SINGER','JACK SOUDERS','MORGAN WESTMORELAND']

contests['CITY OF HUNTINGTON BEACH\nMember, City Council'].winner = ['PAT BURNS','TONY STRICKLAND','GRACEY VAN DER MARK','CASEY MCKEON']

In [ ]:
# construct the dict of dicts of assertions for each contest
Assertion.make_all_assertions(contests)

In [ ]:
audit.check_audit_parameters(contests)